In [17]:
import torch
import torch.nn as nn
import numpy as np
import pylab as plt
import sys, os
parentdir  = os.path.abspath(os.path.join('..'))
print(parentdir)
sys.path.insert(1, os.path.join(parentdir, 'dataloader'))
sys.path.insert(1, os.path.join(parentdir, 'models'))

/Users/bashit.a/Documents/Alzheimer/Codes/ML


In [54]:
### File Locations
column_names= {"Diffuse_Plaque":1. , "Tissue":0.}   # "Diffuse_Plaque"   "Neurofibrillary_Tangle_(tau)"    "Neuritic_Plaque"  "Tissue"
Excel_File  = "/Users/bashit.a/Documents/Alzheimer/Mar-2023/Mar-2023-Samples.xlsx"   # "/home/bashit.a/Codes/ML/Mar-2023-Samples.xlsx"   "/Users/bashit.a/Documents/Alzheimer/Mar-2023/Mar-2023-Samples.xlsx"
sheet       = 'Mar-2023-Samples'
BNL_dir     = '/Volumes/HDD/BNL-Data/Mar-2023'      # '/Volumes/HDD/BNL-Data/Mar-2023'         '/scratch/bashit.a/BNL-Data/Mar-2023'
sub_dir     = "CSV_Conv-8-point"  # CSV_Conv-8-point  CSV
val_files   = ["1948 V1-roi0_0_0_masked.h5"] # None ["1948_HIPPO-roi1_0_0_masked_intp.h5", "2428-roi1_0_0_masked_intp.h5"]


# loading dataset and model from directory
from Transformer import Transformer
from lesions import get_dataloaders_fixed_val_files

# get dataset
qvalue_lower_bound = 0.66
qvalue_upper_bound = 1.46
classes = ['0','1']

# Perform training and validation by checking relative loss on a set of data that was not used for training, and report this loss
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
lidx = np.argmin(qgrid2 < qvalue_lower_bound)   # qvalue = 0.7,  idx = 190
uidx = np.argmin(qgrid2 < qvalue_upper_bound)   # qvalue = 1.46, idx = 342
input_dim = (uidx - lidx)                              # (342-190) = 152


n_heads        = 1
hidden_dim     = 32
N              = 1
dropout        = 0

seq_len        = 2
assert input_dim%seq_len ==0 , f'q value range is not divisible by {seq_len} '
embedding_dim  = input_dim//seq_len

In [57]:
device         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = '../runs/model_20240117_175102'             # path where model is saved

model = Transformer(n_heads, seq_len, embedding_dim, hidden_dim, N, dropout)
model.to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [58]:
weights, training_loader, validation_loader = get_dataloaders_fixed_val_files(Excel_File, sheet, BNL_dir, sub_dir, column_names, val_files, lidx=lidx, uidx=uidx)
X_val, y_val = next(iter(validation_loader))
X_val = X_val.to(device)
y_val = y_val.to(device)
# model.encoder.modules_list[0].self_attention.attention_scores[0,0,:,:]

Setting Training Dataset ...
Diffuse_Plaque:1.0 contains 369 samples
Tissue:0.0 contains 2226 samples
Setting Validation Dataset ...
Diffuse_Plaque:1.0 contains 67 samples
Tissue:0.0 contains 96 samples
weight_for_0 : [0.58288413], weight_for_1 : [3.5162601], weights sum :  [2595.], total samoples : [2595.]
training_loader size : 1 ; validation_loader size : 1


In [97]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

model.eval()
model(X_val, None)
def plot(frame):
    attention_scores = model.encoder.modules_list[0].self_attention.attention_scores[frame,0,:,:].to('cpu').detach().numpy()
    plt.close('all')
    plt.imshow(attention_scores)
    plt.colorbar()
    plt.show()

frames = widgets.SelectionSlider(
    options=validation_loader.dataset.frames,
    value=validation_loader.dataset.frames[0],
    description='Frames : ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact(plot, frame=(0,162,1))

interactive(children=(IntSlider(value=81, description='frame', max=162), Output()), _dom_classes=('widget-inte…

<function __main__.plot(frame)>

In [42]:
qvalue_lower_bound = 0.66
qvalue_upper_bound = 1.46
classes = ['0','1']

# Perform training and validation by checking relative loss on a set of data that was not used for training, and report this loss
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
lidx = np.argmin(qgrid2 < qvalue_lower_bound)   # qvalue = 0.7,  idx = 190
uidx = np.argmin(qgrid2 < qvalue_upper_bound)   # qvalue = 1.46, idx = 342
input_dim = (uidx - lidx) 
input_dim

160

In [94]:
tuple(validation_loader.dataset.frames[:67])

(2547,
 2548,
 2549,
 2550,
 2532,
 2533,
 2534,
 2535,
 2536,
 2537,
 2422,
 2423,
 2424,
 2425,
 2426,
 2427,
 2408,
 2409,
 2410,
 2411,
 2412,
 2413,
 2298,
 2299,
 2300,
 2301,
 2302,
 2303,
 2284,
 2285,
 2286,
 2287,
 2288,
 2289,
 2174,
 2175,
 2176,
 2177,
 2178,
 2179,
 2160,
 2161,
 2162,
 2163,
 2164,
 2165,
 2050,
 2051,
 2052,
 2053,
 2054,
 2055,
 2036,
 2037,
 2038,
 2039,
 2040,
 2041,
 1927,
 1928,
 1929,
 1930,
 1913,
 1914,
 1915,
 1916,
 1917)

In [85]:
type(frames.value)

int

In [1]:
import wandb
import random  # for demo script

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/bashit.a/.netrc


True